In [ ]:
import os
import re
import nltk
import pandas as pd
import numpy as np
import json
nltk.download('stopwords')
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
TOTAL = False

directory = "../data"

if not TOTAL:
    directory = "../pre/sample"

data_list = os.listdir(directory)
data_list = list(data_list)

In [2]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', device='cuda')

In [4]:
doc_embeddings = {}
stop_words = set(stopwords.words('portuguese'))
for file in data_list:
    with open(f'{directory}/{file}', 'r') as f:
        text = f.read()
        text = text.lower()
        text = re.sub(r'[^\w0-9- ]+', '', text, flags=re.UNICODE)
        text = [x.strip() for x in text.split() if len(x) > 0 and x not in stop_words]
        text = ' '.join(text)

        doc_embeddings[file] = list(model.encode(text, device='cuda'))
        #print(doc_embeddings[file].shape)

In [17]:
with open("../pre/global/embed.json", 'w') as f:

    json.dump(doc_embeddings, f)

In [ ]:
def get_similar_docs(document_file:str, embed_json_path:str="../pre/global/embed.json", max_recs:int=5, min_sim:float=0.0):

    with open(embed_json_path, 'r') as f:
        embed_dict = json.load(f)

    """ stop_words = set(stopwords.words('portuguese'))
    with open(document_file, 'r') as f:
        text = f.read()
        text = text.lower()
        text = re.sub(r'[^\w0-9- ]+', '', text, flags=re.UNICODE)
        text = [x.strip() for x in text.split() if len(x) > 0 and x not in stop_words]
        text = ' '.join(text)

        doc_embeds = np.array(model.encode(text, device='cuda')) """
    
    ### Simplificando no momento
    doc_embeds = np.array(embed_dict[document_file])
    
    results = []
    for file in embed_dict.keys():
        embed_value = np.array(embed_dict[file])
        similarity = cosine_similarity(doc_embeds.reshape(1, -1), embed_value.reshape(1, -1))
        results.append((file, similarity[0][0]))


    ordered_results = sorted(results, reverse=True, key=lambda x:x[1])

    return ordered_results[1: max_recs+1]

In [ ]:
print(get_similar_docs('4.txt'))